# Maximum Likelihood Estimation (Generic models)

In [ ]:
from __future__ import print_function
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

The data listed in Appendix Table F21.1 were taken from a study by Spector and Mazzeo
(1980), which examined whether a new method of teaching economics, the Personalized
System of Instruction (PSI), significantly influenced performance in later economics courses.
The "dependent variable" used in our application is GRADE, which indicates the whether
a student's grade in an intermediate macroeconomics course was higher than that in the
principles course. The other variables are GPA, their grade point average; TUCE, the score
on a pretest that indicates entering knowledge of the material; and PSI, the binary variable
indicator of whether the student was exposed to the new teaching method. (Spector and
Mazzeo's specific equation was somewhat different from the one estimated here.)

In [ ]:
from __future__ import print_function
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

The ``Spector`` dataset is distributed with ``statsmodels``. You can access a vector of values for the dependent variable (``endog``) and a matrix of regressors (``exog``) like this:

In [ ]:
data = sm.datasets.spector.load_pandas()
exog = data.exog
endog = data.endog
print(sm.datasets.spector.NOTE)
print(data.exog.head())

Them, we add a constant to the matrix of regressors:

In [ ]:
exog = sm.add_constant(exog, prepend=True)

To create your own Likelihood Model, you simply need to overwrite the loglike method.

In [ ]:
class MyProbit(GenericLikelihoodModel):
    def loglike(self, params):
        exog = self.exog
        endog = self.endog
        q = 2 * endog - 1
        return stats.norm.logcdf(q*np.dot(exog, params)).sum()

Estimate the model and print a summary:

In [ ]:
sm_probit_manual = MyProbit(endog, exog).fit()
print(sm_probit_manual.summary())

Compare your Probit implementation to ``statsmodels``' "canned" implementation:

In [ ]:
sm_probit_canned = sm.Probit(endog, exog).fit()

In [ ]:
print(sm_probit_canned.params)
print(sm_probit_manual.params)

In [ ]:
print(sm_probit_canned.cov_params())
print(sm_probit_manual.cov_params())

Notice that the ``GenericMaximumLikelihood`` class provides automatic differentiation, so we didn't have to provide Hessian or Score functions in order to calculate the covariance estimates.



## Example 2: Negative Binomial Regression for Count Data

Consider a negative binomial regression model for count data with
log-likelihood (type NB-2) function expressed as:

$$
    \mathcal{L}(\beta_j; y, \alpha) = \sum_{i=1}^n y_i ln 
    \left ( \frac{\alpha exp(X_i'\beta)}{1+\alpha exp(X_i'\beta)} \right ) -
    \frac{1}{\alpha} ln(1+\alpha exp(X_i'\beta)) + ln \Gamma (y_i + 1/\alpha) - ln \Gamma (y_i+1) - ln \Gamma (1/\alpha)
$$

with a matrix of regressors $X$, a vector of coefficients $\beta$,
and the negative binomial heterogeneity parameter $\alpha$. 

Using the ``nbinom`` distribution from ``scipy``, we can write this likelihood
simply as:


In [ ]:
import numpy as np
from scipy.stats import nbinom

In [ ]:
def _ll_nb2(y, X, beta, alph):
    mu = np.exp(np.dot(X, beta))
    size = 1/alph
    prob = size/(size+mu)
    ll = nbinom.logpmf(y, size, prob)
    return ll

### New Model Class

We create a new model class which inherits from ``GenericLikelihoodModel``:

In [ ]:
from statsmodels.base.model import GenericLikelihoodModel

In [ ]:
class NBin(GenericLikelihoodModel):
    def __init__(self, endog, exog, **kwds):
        super(NBin, self).__init__(endog, exog, **kwds)
        
    def nloglikeobs(self, params):
        alph = params[-1]
        beta = params[:-1]
        ll = _ll_nb2(self.endog, self.exog, beta, alph)
        return -ll 
    
    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        self.exog_names.append('alpha')
        if start_params == None:
            # Reasonable starting values
            start_params = np.append(np.zeros(self.exog.shape[1]), .5)
            # intercept
            start_params[-2] = np.log(self.endog.mean())
        return super(NBin, self).fit(start_params=start_params, 
                                     maxiter=maxiter, maxfun=maxfun, 
                                     **kwds)

Two important things to notice: 

+ ``nloglikeobs``: This function should return one evaluation of the negative log-likelihood function per observation in your dataset (i.e. rows of the endog/X matrix). 
+ ``start_params``: A one-dimensional array of starting values needs to be provided. The size of this array determines the number of parameters that will be used in optimization.
   
That's it! You're done!

### Usage Example

The [Medpar](http://vincentarelbundock.github.com/Rdatasets/doc/COUNT/medpar.html)
dataset is hosted in CSV format at the [Rdatasets repository](http://vincentarelbundock.github.com/Rdatasets). We use the ``read_csv``
function from the [Pandas library](http://pandas.pydata.org) to load the data
in memory. We then print the first few columns: 


In [ ]:
import statsmodels.api as sm

In [ ]:
medpar = sm.datasets.get_rdataset("medpar", "COUNT", cache=True).data

medpar.head()

The model we are interested in has a vector of non-negative integers as
dependent variable (``los``), and 5 regressors: ``Intercept``, ``type2``,
``type3``, ``hmo``, ``white``.

For estimation, we need to create two variables to hold our regressors and the outcome variable. These can be ndarrays or pandas objects.

In [ ]:
y = medpar.los
X = medpar[["type2", "type3", "hmo", "white"]].copy()
X["constant"] = 1

Then, we fit the model and extract some information: 

In [ ]:
mod = NBin(y, X)
res = mod.fit()

 Extract parameter estimates, standard errors, p-values, AIC, etc.:

In [ ]:
print('Parameters: ', res.params)
print('Standard errors: ', res.bse)
print('P-values: ', res.pvalues)
print('AIC: ', res.aic)

As usual, you can obtain a full list of available information by typing
``dir(res)``.
We can also look at the summary of the estimation results.

In [ ]:
print(res.summary())

### Testing